In [1]:
import torch
import torch.functional as F

# 太阳能板阵列计算框架

## 函数定义总览
接下来定义各种函数，最终输出一个函数`eff_panel`, 描述一个太阳能接收板某一时刻的输出功率。
输入:
- 以接受塔为中心的横纵坐标(x,y)
- 太阳相对阵列的高度角$\alpha_s$, 方位角$\gamma_s$
- 太阳能接收板的面积$A$
- 太阳能接收板到接受塔的水平距离$d$, 太阳能接收板和接受塔的高度$h1, h2$
- 海拔高度$H$

输出:
- 太阳能接收板的输出功率$E_{field}$

## 符号定义
- $A$: 太阳能接收板的面积
- $d$: 太阳能接收板到接受塔的水平距离
- $h1, h2$: 太阳能接收板和接受塔的高度
- $H$: 海拔高度
- $x, y$: 以接受塔为中心的横纵坐标
- $\alpha_s$: 太阳相对阵列的高度角
- $\gamma_s$: 太阳相对阵列的方位角

- DNI: 太阳直射辐照度, 一个用于计算太阳能接收板输出功率的参数
- $\eta_{sb}$: 阴影遮挡下的效率. 考虑了太阳能板之间的阴影遮挡, 以及太阳能板的倾角和方位角对太阳能接收板的阴影遮挡的影响
- $\eta_{cos}$: 太阳能接收板的cosine效率, 考虑了太阳光入射角对太阳能接收板的影响
- $\eta_{at}$: 太阳能接收板的大气效率, 考虑了大气对太阳光的吸收和散射. 表达式为:
$$\eta_{at} = 0.99321-1.1176\times10^{-4}d_{HR}+1.97\times10^{-8}d_{HR}^2$$
其中$d_{HR}$为反射镜中心到集热器中心的距离, 单位为m.
- $\eta_{trunc}$: 太阳能接收板的截断效率, 考虑了太阳能接收板的边缘效应. 这部分损失是由于吸收器无法完全吸收反射光.
- $\eta_{ref}$: 镜面反射率. 考虑为常数, 取0.92.



## 函数数学表达式

### $DNI$ 与阵列无关
$$DNI = G_0[a+bexp(-\frac{c}{sin\alpha_s})]$$
其中$G_0$为太阳常数, $a, b, c$计算如下:
$$a = 0.4237 - 0.00821(6 - H)^2$$
$$b = 0.5055 + 0.00595(6.5 - H)^2$$
$$c = 0.2711 + 0.01858(2.5 - H)^2$$

### $E_{field}$ 与阵列有关
$$E_{field} = DNI \cdot \sum_{i=1}^{N}A_i\eta_{sb}\eta_{cos}\eta_{at}\eta_{trunc}\eta_{ref}$$
其中$A_i$为太阳能接收板的面积, $N$为太阳能接收板的数量.

### $\eta_{sb}$ 与阵列有关
🤯尚未解决, 计划使用最大理论面积/面积和计算.
### $\eta_{cos}$ 与阵列有关
$$\eta_{cos} = cos(\theta)$$
其中$\theta$为太阳光入射角, 计算如下:
🤯$\theta$的计算尚未解决.
### $\eta_{at}$ 与阵列有关
$$\eta_{at} = 0.99321-1.1176\times10^{-4}d_{HR}+1.97\times10^{-8}d_{HR}^2$$
已经给定.
### $\eta_{trunc}$
🤯这一步需要锥型光线假设, 尚未解决.
### $\eta_{ref}$ 与阵列有关
已经给定,常数.
### $d_{HR}$ 与阵列有关
$$d_{HR} = \sqrt{(x-x_{HR})^2+(y-y_{HR})^2+(h_1-h_2)^2}$$
直线距离.


## 功率计算
在模拟阵列的功率时,为简化模型,我们只是用五个时间点的功率值来估计一天的功率值. 五个时间点分别为:
9:00、10:30、12:00、13:30、15:00
在估计一年的功率时,我们使用每个月21日的功率值来估计该月的功率值.

### 数值计算任务
根据任务需求, 我们需要计算**12**天, 每天**5**个时间点的功率值, 包括这**12*5**个时间点的$\eta_{sb}$, $\eta_{cos}$, $\eta_{at}$, $\eta_{trunc}$的值.

## 阵列优化
### 优化目标
优化目标为: 使得阵列的单位面积平均功率最大.
主要约束条件为:
- 阵列的总面积不超过给定的面积
- 阵列的总功率需要超过给定的功率
- 还有吗?
次要约束条件为:
- 吸收塔周围150m范围内不能部署阵列
- 反射镜的高度不能超过镜面高度的一半
- 还有吗?

### 优化方法
> 以下是暂定方法

#### 超参数, 参数, 变量
在前面的问题中, 仅将阵列的位置作为可训练的参数. 将阵列的高度和面积作为超参数.
| 变量 | 参数 | 超参数 | 固定参数 |
| --- | --- | --- | --- |
| 太阳位置参数 | $x, y$ | 阵列数量$n$, 阵列面积$A$, 阵列高度$h_1$ | 略 |

在后续的问题中, 将阵列的位置和高度, 还有阵列的面积作为可训练的参数. 将阵列的数量作为超参数.
| 变量 | 参数 | 超参数 | 固定参数 |
| --- | --- | --- | --- |
| 太阳位置参数 | $x, y$, 阵列面积$A$, 阵列高度$h_1$ | 阵列数量$n$ |  |

#### 优化算法
目前有两个思路:
- 使用遗传算法
- 使用梯度下降算法

In [ ]:
class reflect_matrix(torch.nn.Module):
    def __init__(self, num_panel, trainning_dict, initial_parameters = None, initial_areas = None, initial_heights = None):
        """
        num_panel: number of panels
        parameters: the coordinates of the reflect matrix, (x, y)
        areas: the areas of each panel, can be different
        heights: the heights of each panel, can be different
        """
        super(reflect_matrix, self).__init__()
        self.num_panel = num_panel
        if initial_parameters:
            self.posit = torch.nn.Parameter(torch.tensor(initial_parameters))
        else:
            self.posit = torch.nn.Parameter(torch.randn(num_panel, 2))
        if initial_areas:
            self.areas = torch.nn.Parameter(torch.tensor(initial_areas))
        else:
            self.areas = torch.nn.Parameter(torch.ones(num_panel))
        if initial_heights:
            self.heights = torch.nn.Parameter(torch.tensor(initial_heights))
        else:
            self.heights = torch.nn.Parameter(torch.ones(num_panel))

        for name, value in trainning_dict.items():
            if name == 'posit':
                self.posit.requires_grad = value
            elif name == 'areas':
                self.areas.requires_grad = value
            elif name == 'heights':
                self.heights.requires_grad = value
            else:
                raise ValueError('The name of the trainning_dict is not correct.')
        

    def cal_efficency(self, input):
        """
        input: the input contains the information of sun light, specifically
        - input[:, 0]: 太阳光方位角
        - input[:, 1]: 太阳光高度角
        - input[:, 2]: DNI
        """
        def cal_eta_sb():
            #calculate the eta_sb
            return None

        def cal_eta_cos():
            #calculate the eta_cos
            return None
        
        def cal_eta_at():
            #calculate the eta_at
            return None
        
        def cal_eta_trunc():
            #calculate the eta_trunc
            return None
        
        def cal_eta_ref():
            #calculate the eta_ref
            return None
        
        
        
        # calculate eta
        eta_sb = cal_eta_sb()
        eta_cos = cal_eta_cos()
        eta_at = cal_eta_at()
        eta_trunc = cal_eta_trunc()
        eta_ref = cal_eta_ref()

        eta = eta_sb * eta_cos * eta_at * eta_trunc * eta_ref

        # calculate the efficiency per unit area
        E_field = torch.sum(input[:, 2] * eta)

        # penalty
        ## the sum of the efficiency should be larger than 0.5
        lower_limit = torch.tensor(0.5)
        lower_limit_penalty = torch.min(torch.tensor(0), E_field - lower_limit)

        return E_field

